<a href="https://colab.research.google.com/github/hammad558/OPENAI_SDK_OPENROUTER/blob/main/OPENAI_SDK_OPENROUTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use OpenRouter With OpenAI Agents SDK

Setup Prerequisite:

1. [Signup at OpenRouter](https://openrouter.ai/)
2. [Create an API Key](https://openrouter.ai/settings/keys)
2. Select a Free Model (you can continue as we are using a free model here)

## Free and Paid Models

The OpenRouter supports the latest DeepSeek V3 0324 and 50+ other models for free. Most of them support the defacto standard: OpenAI Chat Completion API.


If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

**See all Models List: https://openrouter.ai/models**

Note: OpenRouter do not charge anything extra at inference time.

## Rate Limiting and Crediting

There are a few rate limits that apply to certain types of requests, regardless of account status:

- Free limit: If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

If your account has a negative credit balance, you may see 402 errors, including for free models. Adding credits to put your balance above zero allows you to use those models again.

[Reference](https://openrouter.ai/docs/api-reference/limits)

## Install OpenAI Agents Dep.

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [3]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [7]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "google/gemma-3-1b-it:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [8]:
BASE_URL

'https://openrouter.ai/api/v1'

In [9]:
import requests
import json

response = requests.post(
  url=f"https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "Hi, i have 1000 pkr i want you to convert into usd"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1750054351-fauJJZ6iSglxlqB9RbMJ', 'provider': 'Chutes', 'model': 'google/gemma-3-1b-it:free', 'object': 'chat.completion', 'created': 1750054351, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': " should I convert to USD?\n\nI'm sorry, but as an AI, I cannot directly convert PKR to USD. I do not have real-time access to currency exchange rates.\n\n**However, here's how you can easily convert your PKR to USD:**\n\n1. **Use Online Currency Converters:** The easiest way is to use a reliable online converter. Here are a few popular options:\n   * **Google:** [https://www.google.com/finance/convert/pkr-usd](https://www.google.com/finance/convert/pkr-usd)\n   * **XE.com:** [https://www.xe.com/currencyconverter/convert/?exchange=pkr-usd](https://www.xe.com/currencyconverter/convert/?exchange=pkr-usd)\n   * **Wise:** [https://www.wise.io/](https://www.wise.io/) (Wise is a good option for 

In [10]:
data = response.json()
data['choices'][0]['message']['content']

" should I convert to USD?\n\nI'm sorry, but as an AI, I cannot directly convert PKR to USD. I do not have real-time access to currency exchange rates.\n\n**However, here's how you can easily convert your PKR to USD:**\n\n1. **Use Online Currency Converters:** The easiest way is to use a reliable online converter. Here are a few popular options:\n   * **Google:** [https://www.google.com/finance/convert/pkr-usd](https://www.google.com/finance/convert/pkr-usd)\n   * **XE.com:** [https://www.xe.com/currencyconverter/convert/?exchange=pkr-usd](https://www.xe.com/currencyconverter/convert/?exchange=pkr-usd)\n   * **Wise:** [https://www.wise.io/](https://www.wise.io/) (Wise is a good option for international transfers)\n\n2. **Use a Currency Exchange Service:** Many banks and financial institutions offer currency exchange services.\n\n**Important Considerations:**\n\n* **Exchange Rate Fluctuations:** The exchange rate changes constantly. The rate you get will vary depending on the provider a

## 2. Using OpenAI Agents SDK

In [11]:
import asyncio
from openai import AsyncOpenAI # chat completions
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

set_tracing_disabled(disabled=True) # Open AI Tracing == Disable

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)


async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="LahoreTA",
        instructions="You only respond in english.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent, # starting agent
        "What is your name?.", # request
    )
    print(result.final_output)


asyncio.run(main())


I am Gemma, a large language model created by the Gemma team at Google DeepMind.


In [12]:
response.json()

{'id': 'gen-1750054351-fauJJZ6iSglxlqB9RbMJ',
 'provider': 'Chutes',
 'model': 'google/gemma-3-1b-it:free',
 'object': 'chat.completion',
 'created': 1750054351,
 'choices': [{'logprobs': None,
   'finish_reason': 'stop',
   'native_finish_reason': 'stop',
   'index': 0,
   'message': {'role': 'assistant',
    'content': " should I convert to USD?\n\nI'm sorry, but as an AI, I cannot directly convert PKR to USD. I do not have real-time access to currency exchange rates.\n\n**However, here's how you can easily convert your PKR to USD:**\n\n1. **Use Online Currency Converters:** The easiest way is to use a reliable online converter. Here are a few popular options:\n   * **Google:** [https://www.google.com/finance/convert/pkr-usd](https://www.google.com/finance/convert/pkr-usd)\n   * **XE.com:** [https://www.xe.com/currencyconverter/convert/?exchange=pkr-usd](https://www.xe.com/currencyconverter/convert/?exchange=pkr-usd)\n   * **Wise:** [https://www.wise.io/](https://www.wise.io/) (Wise 